In [29]:
import numpy as np
from torchvision import datasets, transforms    
from torchvision.utils import make_grid
from torch import nn

In [35]:
transform = transforms.Compose([
    transforms.ToTensor()  
])

In [ ]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

100.0%
100.0%
100.0%
100.0%


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
model = CNN()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
num_params

421642

In [49]:
# Training the model
import torch
from torch import optim
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
optimizer = optim.SGD(model.parameters())
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


100%|██████████| 938/938 [00:12<00:00, 75.52it/s]


Epoch [1/5], Loss: 2.2911


100%|██████████| 938/938 [00:10<00:00, 87.75it/s]


Epoch [2/5], Loss: 2.2600


100%|██████████| 938/938 [00:10<00:00, 89.89it/s]


Epoch [3/5], Loss: 2.1819


100%|██████████| 938/938 [00:10<00:00, 88.45it/s]


Epoch [4/5], Loss: 1.9115


100%|██████████| 938/938 [00:12<00:00, 72.46it/s]

Epoch [5/5], Loss: 1.3106


In [51]:
# Validate the model
from sklearn.metrics import accuracy_score
def validate_model(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy
# Validate the model
val_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
accuracy = validate_model(model, val_loader)
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Accuracy: 0.7885
